<a href="https://colab.research.google.com/github/yosuke318/python_notebook/blob/main/dataclass_pydantic_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pprint

In [15]:
from dataclasses import dataclass
@dataclass(slots=True) # __slots__を使用することでメモリ使用量を大幅に削減できます
class Product:
  id: int
  name: str
  price:float

product1 = Product(id=1, name="apple", price=100.0)
pprint.pprint(product1)

product2 = Product(id="2", name="greenapple", price="1000")
pprint.pprint(product2)

Product(id=1, name='apple', price=100.0)
Product(id='2', name='greenapple', price='1000')


In [23]:
from pydantic.dataclasses import rebuild_dataclass
from pydantic import ValidationError, Field, BaseModel


class Book(BaseModel):
  title: str=Field(strict=True)
  pages: int=Field(strict=True)

try:
  book = Book(title="111", pages="123")
  pprint.pprint(book)
except ValidationError as e:
  pprint.pprint(e)

1 validation error for Book
pages
  Input should be a valid integer [type=int_type, input_value='123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_type


In [31]:
from datetime import date
from typing import Optional, List
from pydantic import HttpUrl, Field

class TecnicalBook(Book):
  isbn: str = Field(strict=True, pattern=r'^(?:\d{10}|\d{13})$')
  author: str = Field(strict=True)
  publication_date: date = Field(strict=True)
  programming_language: Optional[str]=Field(default=None, strict=True)
  topics: List[str]=Field(default_factory=list)
  edition: Optional[int] = Field(defalut=1, strict=True, ge=1)
  publisher: str = Field(strict=True)
  website: Optional[HttpUrl] = Field(defalut=None)

  class Config:
    extra = 'forbid'

In [32]:
from datetime import date

tech_book = TecnicalBook(
    title="Python高速プログラミング",
    pages=300,
    isbn="9784873119120",
    author="ミーカ・ルーソ",
    publication_date=date(2023, 1, 1),
    programming_language="Python",
    topics=["最適化", "並列処理", "非同期プログラミング"],
    edition=2,
    publisher="オライリージャパン",
    website="https://www.oreilly.co.jp/books/9784873119120/"
)

pprint.pprint(tech_book)


TecnicalBook(title='Python高速プログラミング', pages=300, isbn='9784873119120', author='ミーカ・ルーソ', publication_date=datetime.date(2023, 1, 1), programming_language='Python', topics=['最適化', '並列処理', '非同期プログラミング'], edition=2, publisher='オライリージャパン', website=HttpUrl('https://www.oreilly.co.jp/books/9784873119120/'))


In [37]:
from pydantic import BaseModel, validator,conint, constr, UrlConstraints
from typing import List

class Person(BaseModel):
  name: constr(strip_whitespace=True, min_length=1)
  age: conint(ge=10, le=100)





<ipython-input-37-1be3c8f13ccf>:12: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator('experiences')


In [42]:
class Engineer(Person):
    skills: List[str]
    experiences: List[int]

    @validator('experiences')
    @classmethod
    def check_experiences(cls, v, values):
        required_experience = {
            'Python': 3,
            'Java': 3,
            'C++': 3,
            # 他の言語と必要な経験年数を追加
        }
        for skill in values.get('skills', []):
            if skill in required_experience and any(exp < required_experience[skill] for exp in v):
                raise ValueError(f'{skill} engineer must have at least {required_experience[skill]} years of experience')
        return v

<ipython-input-42-38f51d72ea60>:5: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator('experiences')


In [43]:
try:
    engineer = Engineer(name="John Doe", age=30, skills=["Python"], experiences=[2])
except ValidationError as e:
    print(e)

1 validation error for Engineer
experiences
  Value error, Python engineer must have at least 3 years of experience [type=value_error, input_value=[2], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


In [44]:
try:
    engineer = Engineer(name="John Doe", age=30, skills=["Python"], experiences=[3])
except ValidationError as e:
    print(e)

In [45]:
try:
    engineer = Engineer(name="John Doe", age=30, skills=["Java"], experiences=[2])
except ValidationError as e:
    print(e)

1 validation error for Engineer
experiences
  Value error, Java engineer must have at least 3 years of experience [type=value_error, input_value=[2], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
